In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import warnings
sys.path.append('../')  # |返回notebook的上一级目录
warnings.filterwarnings("ignore")

E:\software\Anaconda3\envs\tf2.0\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
from batcher import *
from utils.saveLoader import Vocab
from utils.config import VOCAB_PAD

In [3]:
vocab = Vocab(VOCAB_PAD)
vocab.word_to_id("你好"), vocab.id_to_word(519)

(4, '找到')

In [4]:
%run ../utils/params

In [13]:
gen = iter(batcher(vocab, params))

In [30]:
batch[1].keys()

dict_keys(['dec_input', 'dec_target', 'dec_len', 'abstract', 'sample_decoder_pad_mask'])

In [4]:
from preprocess import Preprocess
proc = Preprocess()

创建一个预处理器


In [5]:
ori_sentence = "道奇,锋哲,昨晚看到一台车车牌是绿色的？这是什么牌？"
proc_sentence = proc.sentence_proc(ori_sentence).split(" ")


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Light\AppData\Local\Temp\jieba.cache
Loading model cost 0.728 seconds.
Prefix dict has been built succesfully.


### article_to_ids 演示
input:
- article_words: list 分好词的句子
- vocab: Vocab词表类

output:
- ids: list 
- oovs: list 

In [6]:
print("article", proc_sentence)
ids, oovs = article_to_ids(proc_sentence, vocab)
print("indexs:", ids)
print("oovs:",oovs)

article ['道奇', '，', '锋哲', '，', '昨晚', '看到', '一台', '车', '车牌', '是', '绿色', '的', '？', '这是', '什么', '牌', '？']
indexs: [7164, 0, 32247, 0, 2497, 435, 2405, 33, 4146, 4, 2103, 1, 5, 208, 27, 2922, 5]
oovs: ['锋哲']


### abstract_to_ids
input:
- abstract_words: 
- vocab: 
- article_oovs: 

output:
- ids: article_oovs

In [7]:
abstract_words = ["这是", "车辆", "道奇", "，", "锋哲"]
abstract_ids = abstract_to_ids(abstract_words, vocab, oovs)
print("vocab最大索引:", max(vocab.id2word.keys()))
# 锋哲是oov词，因为vocab最大索引只到32246
abstract_ids

vocab最大索引: 32246


[208, 104, 7164, 0, 32247]

### output_to_words
input:
- id_list: 模型预测的index list
- vocab: 
- article_oovs: 

output:
- ids: words

In [8]:
abstract_ids

[208, 104, 7164, 0, 32247]

In [9]:
output_to_words(abstract_ids, vocab, oovs)  # 其实=abstract_words

['这是', '车辆', '道奇', '，', '锋哲']

### get_dec_inp_targ_seqs
input:
- sequence
- maxlen
- start_id
- stop_id

output:
- inp 作为decoder的输入
- target 用来计算损失

In [214]:
max_len = 45
start_id = vocab.word2id[vocab.START_DECODING]
stop_id = vocab.word2id[vocab.STOP_DECODING]
start_id, stop_id

(32243, 32244)

In [215]:
inp, target = get_dec_inp_targ_seqs(proc_sentence, max_len, start_id, stop_id)
print("proc_sentence:", proc_sentence)
print(inp)
print(target)

proc_sentence: ['道奇', '，', '锋哲', '，', '昨晚', '看到', '一台', '车', '车牌', '是', '绿色', '的', '？', '这是', '什么', '牌', '？']
[32243, '道奇', '，', '锋哲', '，', '昨晚', '看到', '一台', '车', '车牌', '是', '绿色', '的', '？', '这是', '什么', '牌', '？']
['道奇', '，', '锋哲', '，', '昨晚', '看到', '一台', '车', '车牌', '是', '绿色', '的', '？', '这是', '什么', '牌', '？', 32244]


### example_generator()
每次只产生一个样本

根据mode是否为train来决定产生的样本是训练集样本还是测试集样本

In [12]:
# 运行这个文件会获得params变量
%run ../utils/params.py

In [13]:
params

{'mode': 'train',
 'max_enc_len': 460,
 'max_dec_len': 52,
 'batch_size': 32,
 'epochs': 1,
 'vocab_path': 'E:\\GitHub\\QA-abstract-and-reasoning\\data\\wv\\vocab_index_pad.txt',
 'learning_rate': 0.01,
 'adagrad_init_acc': 0.1,
 'max_grad_norm': 0.8,
 'vocab_size': 32247,
 'beam_size': 3,
 'embed_size': 300,
 'enc_units': 256,
 'dec_units': 256,
 'attn_units': 128,
 'train_seg_x_dir': 'E:\\GitHub\\QA-abstract-and-reasoning\\data\\train_seg_x.csv',
 'train_seg_y_dir': 'E:\\GitHub\\QA-abstract-and-reasoning\\data\\train_seg_y.csv',
 'test_seg_x_dir': 'E:\\GitHub\\QA-abstract-and-reasoning\\data\\test_seg_x.csv',
 'checkpoints_save_steps': 2,
 'min_dec_steps': 4,
 'max_train_steps': 2592,
 'train_pickle_dir': 'E:\\GitHub\\QA-abstract-and-reasoning\\data\\dataset',
 'save_batch_train_data': False,
 'load_batch_train_data': False}

In [14]:
max_enc_len = params["max_enc_len"]
max_dec_len = params["max_dec_len"]
mode = params["mode"]
batch_size = params["batch_size"]
max_enc_len, max_dec_len, mode, batch_size

(460, 52, 'train', 32)

In [15]:
output = example_generator(params, vocab, max_enc_len, max_dec_len, mode, batch_size)

In [16]:
ds = batcher(vocab, params)

### 构建数据集

In [17]:
dataset_1 = tf.data.TextLineDataset(params["train_seg_x_dir"])
dataset_2 = tf.data.TextLineDataset(params["train_seg_y_dir"])

train_dataset = tf.data.Dataset.zip((dataset_1, dataset_2))
train_dataset = train_dataset.shuffle(10, reshuffle_each_iteration=True).repeat()

In [18]:
next(iter(dataset_1)).numpy().decode()

'方向机 重 ， 助力 泵 ， 方向机 都 换 了 还是 一样 新 的 都 换 了 助力 泵 ， 方向机 换 了 方向机 带 的 有 有 助力 就是 重 ， 这车 要 匹配 吧 不 需要 你 这 是 更换 的 部件 有 问题 跑 快 了 还好 点 ， 就 倒车 重 的 很 。 是 非常 重 吗 是 的 ， 累人 我 觉得 也 是 ， 可是 车主 是 以前 没 这么 重 ， 选 吧 助理 泵 换 了 不行 ， 又 把 放 向 机换 了 ， 现在 还 这样 就 不 知道 咋 和 车主 解释 。'

In [19]:
next(iter(dataset_2)).numpy().decode()

'随时 联系'

In [20]:
raw_record = next(iter(train_dataset))

In [21]:
article = raw_record[0].numpy().decode("utf-8")
article

'昌河Q35 音响 怎么 拆装 发个 中控 看看 圈起来 的 面板 吗 是 的 翘出来 不行 如图所示 没有 螺丝 了 新车 就是 紧翘 的 要 断 了 就是 不 出来 绿色 圈 的 先 拿 下来 上边 有 螺丝 你 没有 拆完 吗 就是 内部 卡扣 固定'

In [22]:
start_decoding = vocab.word_to_id(Vocab.START_DECODING)
stop_decoding = vocab.word_to_id(Vocab.STOP_DECODING)
start_decoding, stop_decoding

(32243, 32244)

In [23]:
article_words = article.split()[:max_enc_len]
enc_len = len(article_words)
print(article_words)
print(enc_len)

['昌河Q35', '音响', '怎么', '拆装', '发个', '中控', '看看', '圈起来', '的', '面板', '吗', '是', '的', '翘出来', '不行', '如图所示', '没有', '螺丝', '了', '新车', '就是', '紧翘', '的', '要', '断', '了', '就是', '不', '出来', '绿色', '圈', '的', '先', '拿', '下来', '上边', '有', '螺丝', '你', '没有', '拆完', '吗', '就是', '内部', '卡扣', '固定']
46


In [24]:
sample_encoder_pad_mask = [1 for _ in range(enc_len)]
print(sample_encoder_pad_mask)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [25]:
enc_input = [vocab.word_to_id(w) for w in article_words]
enc_input_extend_vocab, article_oovs = article_to_ids(article_words, vocab)
print(enc_input)
print(enc_input_extend_vocab)
print(article_oovs)

[32246, 1192, 68, 987, 1635, 992, 92, 8768, 1, 1251, 14, 4, 1, 8858, 169, 6029, 15, 229, 3, 294, 29, 32246, 1, 46, 719, 3, 29, 11, 176, 2103, 2464, 1, 128, 570, 453, 2651, 6, 229, 12, 15, 6397, 14, 29, 343, 1401, 513]
[32247, 1192, 68, 987, 1635, 992, 92, 8768, 1, 1251, 14, 4, 1, 8858, 169, 6029, 15, 229, 3, 294, 29, 32248, 1, 46, 719, 3, 29, 11, 176, 2103, 2464, 1, 128, 570, 453, 2651, 6, 229, 12, 15, 6397, 14, 29, 343, 1401, 513]
['昌河Q35', '紧翘']


In [26]:
abstract = raw_record[1].numpy().decode("utf-8")
abstract_words = abstract.split()
abs_ids = [vocab.word_to_id(w) for w in abstract_words]
abs_ids_extend_vocab = abstract_to_ids(abstract_words, vocab, article_oovs)
print(abstract)
print(abstract_words)
print(abs_ids)  # 直接根据词表映射
print(abs_ids_extend_vocab)  # 输出没有oov词，则于abs_ids一致

这个 面板 就是 内部 卡子 固定 ， 没有 固定 螺丝
['这个', '面板', '就是', '内部', '卡子', '固定', '，', '没有', '固定', '螺丝']
[8, 1251, 29, 343, 1292, 513, 0, 15, 513, 229]
[8, 1251, 29, 343, 1292, 513, 0, 15, 513, 229]


In [27]:
dec_input, target = get_dec_inp_targ_seqs(abs_ids, max_dec_len, start_decoding, stop_decoding)
_, target = get_dec_inp_targ_seqs(abs_ids_extend_vocab, max_dec_len, start_decoding, stop_decoding)
dec_len = len(dec_input)
print("dec_input:", dec_input)
print("target:", target)
print("dec_len:", dec_len)

dec_input: [32243, 8, 1251, 29, 343, 1292, 513, 0, 15, 513, 229]
target: [8, 1251, 29, 343, 1292, 513, 0, 15, 513, 229, 32244]
dec_len: 11


In [28]:
sample_decoder_pad_mask = [1 for _ in range(dec_len)]
print(sample_decoder_pad_mask, len(sample_decoder_pad_mask))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 11


In [29]:
output = {
                "enc_len": enc_len,
                "enc_input": enc_input,
                "enc_input_extend_vocab": enc_input_extend_vocab,
                "article_oovs": article_oovs,
                "dec_input": dec_input,
                "target": target,
                "dec_len": dec_len,
                "article": article,
                "abstract": abstract,
                "abstract_sents": abstract,
                "sample_decoder_pad_mask": sample_decoder_pad_mask,
                "sample_encoder_pad_mask": sample_encoder_pad_mask
            }

In [30]:
output["article"]

'昌河Q35 音响 怎么 拆装 发个 中控 看看 圈起来 的 面板 吗 是 的 翘出来 不行 如图所示 没有 螺丝 了 新车 就是 紧翘 的 要 断 了 就是 不 出来 绿色 圈 的 先 拿 下来 上边 有 螺丝 你 没有 拆完 吗 就是 内部 卡扣 固定'

In [46]:
ds = batch_generator(example_generator, params,
                                  vocab, params["max_enc_len"],
                                  params["max_dec_len"], params["batch_size"], params["mode"])

In [47]:
n = next(iter(ds))

In [54]:
type(n[0]), type(n[1])

(dict, dict)

In [55]:
n[0].keys()

dict_keys(['enc_input', 'extended_enc_input', 'article_oovs', 'enc_len', 'article', 'max_oov_len', 'sample_encoder_pad_mask'])

In [56]:
n[1].keys()

dict_keys(['dec_input', 'dec_target', 'dec_len', 'abstract', 'sample_decoder_pad_mask'])

In [58]:
vocab.id2word[32245]

'<PAD>'

In [86]:
gen = (x for x in range(1,10,2))

In [78]:
type(gen)

generator

In [94]:
dataset = tf.data.Dataset.from_generator(
        lambda: example_generator(params, vocab, max_enc_len, max_dec_len, mode, batch_size),
        output_types={
            "enc_len": tf.int32,
            "enc_input": tf.int32,
            "enc_input_extend_vocab": tf.int32,
            "article_oovs": tf.string,
            "dec_input": tf.int32,
            "target": tf.int32,
            "dec_len": tf.int32,
            "article": tf.string,
            "abstract": tf.string,
            "abstract_sents": tf.string,
            "sample_decoder_pad_mask": tf.int32,
            "sample_encoder_pad_mask": tf.int32,
        },
        output_shapes={
            "enc_len": [],
            "enc_input": [None],
            "enc_input_extend_vocab": [None],
            "article_oovs": [None],
            "dec_input": [None],
            "target": [None],
            "dec_len": [],
            "article": [],
            "abstract": [],
            "abstract_sents": [],
            "sample_decoder_pad_mask": [None],
            "sample_encoder_pad_mask": [None]
        })

In [95]:
next(iter(dataset))

{'enc_len': <tf.Tensor: id=882, shape=(), dtype=int32, numpy=130>,
 'enc_input': <tf.Tensor: id=880, shape=(130,), dtype=int32, numpy=
 array([ 1352,  2660,  9241,   129,   307,   126,     1,  2725,     2,
           37,     7,   188,     1,  2725,    85,   161,   323,    28,
           85,   161,    12,     1,   328,     4,     9,   488,    30,
           52,   149,     1,   129,     9,    13,    47,     2,   488,
           30,   547,     1,   129,     0,     8,   248,     4,  1255,
         1030,   101,     2,  2904,   129, 32246,   678,     4,     7,
          188,     1,    17,    46,  1255,    14,     4,     1,     0,
            8,   129,     4,   159,   535,   129,     0,   344,   129,
          450,   120,    93,     0,   244,    40,   248,    20,   120,
          642,     0,    62,    29,   505,   856,     1,     2,    12,
           13,   188,    16,   530,   546,     0,     8,     6,   258,
          145,     1,     0,   195,   137,     8,  1423,    11,   118,
            0

In [110]:
gen = ([i,i+1,i+2]for i in range(5) )

In [111]:
for i in gen:print(i)

[0, 1, 2]
[1, 2, 3]
[2, 3, 4]
[3, 4, 5]
[4, 5, 6]


In [208]:
gen = ({"ls":[i,i+1,i+2],
        "num":i,
        "name":"test"+str(i)}  if (i+1)%2==0 else {"ls":[], "num":0, "name":b""}for i in range(5))
ds = tf.data.Dataset.from_generator(lambda: gen, 
                                    output_types = {"ls":tf.int32, 
                                                    "num":tf.int32, 
                                                    "name":tf.string}, 
                                    output_shapes = {"ls":[None], 
                                                    "num":[], 
                                                    "name":[]})

In [195]:
for i in ds:
    for k,v in i.items():
        print(k,v)
    print()

ls tf.Tensor([], shape=(0,), dtype=int32)
num tf.Tensor(0, shape=(), dtype=int32)
name tf.Tensor(b'', shape=(), dtype=string)

ls tf.Tensor([1 2 3], shape=(3,), dtype=int32)
num tf.Tensor(1, shape=(), dtype=int32)
name tf.Tensor(b'test1', shape=(), dtype=string)

ls tf.Tensor([], shape=(0,), dtype=int32)
num tf.Tensor(0, shape=(), dtype=int32)
name tf.Tensor(b'', shape=(), dtype=string)

ls tf.Tensor([3 4 5], shape=(3,), dtype=int32)
num tf.Tensor(3, shape=(), dtype=int32)
name tf.Tensor(b'test3', shape=(), dtype=string)

ls tf.Tensor([], shape=(0,), dtype=int32)
num tf.Tensor(0, shape=(), dtype=int32)
name tf.Tensor(b'', shape=(), dtype=string)



[padded_batch](https://tensorflow.google.cn/api_docs/python/tf/data/Dataset#padded_batch)

In [209]:
ds = ds.padded_batch(batch_size = 2, 
                     padded_shapes={"ls":[2], 
                                    "num":[], 
                                    "name":[]},
                     padding_values={"ls":100, 
                                    "num":-1, 
                                    "name":b"shit"}
                    )

In [216]:
dataset = batch_generator(example_generator, params,
                                  vocab, params["max_enc_len"],
                                  params["max_dec_len"], params["batch_size"], params["mode"])

In [217]:
a = iter(dataset)

In [234]:
next(a)

({'enc_input': <tf.Tensor: id=3632, shape=(32, 290), dtype=int32, numpy=
  array([[   96,     3,   355, ...,    29,    26,  4973],
         [  801,  1378,    22, ..., 32245, 32245, 32245],
         [ 5777,   618,   131, ..., 32245, 32245, 32245],
         ...,
         [  817,   114,  1381, ..., 32245, 32245, 32245],
         [ 6114,   507,  1000, ..., 32245, 32245, 32245],
         [    7,   193,   109, ..., 32245, 32245, 32245]])>,
  'extended_enc_input': <tf.Tensor: id=3634, shape=(32, 290), dtype=int32, numpy=
  array([[   96,     3,   355, ...,    29,    26,  4973],
         [  801,  1378,    22, ..., 32245, 32245, 32245],
         [ 5777,   618,   131, ..., 32245, 32245, 32245],
         ...,
         [  817,   114,  1381, ..., 32245, 32245, 32245],
         [ 6114,   507,  1000, ..., 32245, 32245, 32245],
         [    7,   193,   109, ..., 32245, 32245, 32245]])>,
  'article_oovs': <tf.Tensor: id=3631, shape=(32, 4), dtype=string, numpy=
  array([[b'\xe6\x8d\xa2\xe8\xa7\x92', b

In [235]:
dataset = batcher(vocab, params)

In [236]:
it = iter(dataset)

In [240]:
next(it)

({'enc_input': <tf.Tensor: id=4084, shape=(32, 447), dtype=int32, numpy=
  array([[  275,  1206,   181, ..., 32245, 32245, 32245],
         [ 1163,   114,   801, ...,   486,   457,     2],
         [ 1352,  9563,   409, ..., 32245, 32245, 32245],
         ...,
         [ 5732, 22847, 28922, ..., 32245, 32245, 32245],
         [   19,     0,   109, ..., 32245, 32245, 32245],
         [ 1175,  2448,   706, ..., 32245, 32245, 32245]])>,
  'extended_enc_input': <tf.Tensor: id=4086, shape=(32, 447), dtype=int32, numpy=
  array([[  275,  1206,   181, ..., 32245, 32245, 32245],
         [ 1163,   114,   801, ...,   486,   457,     2],
         [ 1352,  9563,   409, ..., 32245, 32245, 32245],
         ...,
         [ 5732, 22847, 28922, ..., 32245, 32245, 32245],
         [   19,     0,   109, ..., 32245, 32245, 32245],
         [ 1175,  2448,   706, ..., 32245, 32245, 32245]])>,
  'article_oovs': <tf.Tensor: id=4083, shape=(32, 3), dtype=string, numpy=
  array([[b'308687854', b'', b''],
     